In [1]:
#limpio la memoria
rm( list=ls() )  #remove all objects
gc()             #garbage collection

require("data.table")
require("scales")
require("lightgbm")

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,604568,32.3,1292120,69.1,1292120,69.1
Vcells,1087066,8.3,8388608,64.0,1632674,12.5


Loading required package: data.table

Loading required package: scales

Loading required package: lightgbm

Loading required package: R6



In [2]:
#Parametros del script
setwd("C:/_MCD/Labo1/code")  #Establezco el Working Directory
#setwd("~/buckets/b1/")

#cargo el dataset
dataset  <- fread("./datasets/comp1/competencia1_2022.csv")   #donde entreno# FIN Parametros del script

In [3]:
graficar_campo  <- function( campo, periodos_analisis )
{

  #quito de grafico las colas del 5% de las densidades

  tb_quantiles  <- data.table( qmin= numeric(),  qmax= numeric() )

  for( periodo in periodos_analisis )
  {
    qu  <- quantile(  dataset[ foto_mes==periodo , get(campo) ] , prob= c(0.05, 0.95), na.rm=TRUE )
    tb_quantiles  <- rbind( tb_quantiles, use.names=FALSE, as.list(qu) )
  }

  xxmin  <- tb_quantiles[ , min( qmin ) ]
  xxmax  <- tb_quantiles[ , max( qmax ) ]

  yymax  <- 0
  for( per in periodos_analisis )
  {
    den  <- density( dataset[ foto_mes==per, get(campo) ],
                     kernel="gaussian", na.rm=TRUE )

    mayor  <- max( den$y )
    if( mayor > yymax ) yymax <- mayor 
  }

  densidad_A  <- density( dataset[ foto_mes==periodos_analisis[1], get(campo) ],
                          kernel="gaussian", na.rm=TRUE )


  plot( densidad_A,
        col= GLOBAL_colores[1],
        xlim= c( xxmin, xxmax ),
        ylim= c( 0, yymax ),
        main= paste0( campo  ) 
      )

  for( per in 2:length(periodos_analisis) )
  {
    densidad_B  <- density( dataset[ foto_mes==periodos_analisis[ per ], get(campo) ],
                            kernel="gaussian", na.rm=TRUE )

    lines(densidad_B, col= GLOBAL_colores[per], lty=1)
  }
  
  legend(  "topright",  
           legend= periodos_analisis,
           col=GLOBAL_colores, lty=1 )

}

In [4]:
kexperimento <- "comp1-EDA"
#creo la carpeta donde va el experimento
dir.create( paste0( "./exp/", kexperimento, "/"), showWarnings = FALSE )
setwd(paste0( "./exp/", kexperimento, "/"))   #Establezco el Working Directory DEL EXPERIMENTO


In [5]:
#creo la clase_binaria SI={ BAJA+1, BAJA+2 }    NO={ CONTINUA }
dataset[ foto_mes<=202103, 
         clase01 :=  ifelse( clase_ternaria=="CONTINUA", 0, 1 ) ]

dataset[  , foto_mes := as.character( foto_mes ) ]

In [6]:
#Se toman estos campos de otro script que hizo el entrenamiento con unos HP buenos
campos_ordenados <- c("ctrx_quarter","cpayroll_trx","mcaja_ahorro","mpayroll","mcuentas_saldo","mpasivos_margen","mtarjeta_visa_consumo","mprestamos_personales","mrentabilidad_annual","mcuenta_corriente","mactivos_margen","active_quarter","ctarjeta_visa_transacciones","cprestamos_personales","Visa_fechaalta","mcomisiones_mantenimiento","cdescubierto_preacordado","Visa_status","Visa_mpagominimo","Master_fechaalta","cliente_edad","mrentabilidad","Master_Fvencimiento","Visa_msaldototal","numero_de_cliente","Visa_Fvencimiento","cproductos","cliente_antiguedad","ctarjeta_master","Visa_msaldopesos","ccaja_seguridad","Master_mfinanciacion_limite","mcuenta_debitos_automaticos","Master_status","mtransferencias_recibidas","chomebanking_transacciones","ccomisiones_mantenimiento","mautoservicio","ccajas_consultas","mtarjeta_master_consumo","ccomisiones_otras","Visa_cconsumos","Visa_mfinanciacion_limite","mcaja_ahorro_dolares","mttarjeta_visa_debitos_automaticos","mcomisiones_otras","mcomisiones","Visa_mpagospesos","Visa_mconsumospesos","Master_mlimitecompra","ccallcenter_transacciones","thomebanking","ccaja_ahorro","tcallcenter","Visa_mlimitecompra","ctransferencias_recibidas","Master_msaldototal","Master_fultimo_cierre","cmobile_app_trx","mtransferencias_emitidas","ctarjeta_debito_transacciones","mpagomiscuentas","Visa_fultimo_cierre","ctarjeta_visa_debitos_automaticos","Master_msaldopesos","Visa_mconsumosdolares","ctarjeta_visa","ctarjeta_master_transacciones","Visa_mconsumototal","Master_mconsumospesos","mplazo_fijo_dolares","ctarjeta_debito","Master_mpagominimo","Master_mpagospesos","ccuenta_debitos_automaticos","ctransferencias_emitidas","mextraccion_autoservicio","Visa_mpagado","ccajas_transacciones","cextraccion_autoservicio","cpagomiscuentas","Master_cconsumos","Visa_msaldodolares","matm_other","Master_mconsumototal","matm","ccajas_otras","catm_trx","cseguro_vida","ccajas_depositos","ccajas_extracciones","cinversion1","mcheques_depositados","mttarjeta_master_debitos_automaticos","Visa_mpagosdolares","cplazo_fijo","Master_madelantopesos","Master_mpagado","cseguro_accidentes_personales","mforex_sell","catm_trx_other","Master_mpagosdolares","Master_mconsumosdolares","cseguro_vivienda","mcaja_ahorro_adicional","Visa_Finiciomora","minversion2","cinversion2","ctarjeta_master_debitos_automaticos","ccheques_emitidos","Visa_delinquency","Master_msaldodolares","mcheques_emitidos","Master_cadelantosefectivo","Master_delinquency","ccheques_depositados","Visa_madelantopesos","mtarjeta_visa_descuentos","Visa_cadelantosefectivo","Visa_madelantodolares","cforex","cliente_vip","tcuentas","ccuenta_corriente","mcuenta_corriente_adicional","cprestamos_prendarios","mprestamos_prendarios","cprestamos_hipotecarios","mprestamos_hipotecarios","mplazo_fijo_pesos","minversion1_pesos","minversion1_dolares","cseguro_auto","mpayroll2","cpayroll2_trx","cpagodeservicios","mpagodeservicios","ccajeros_propios_descuentos","mcajeros_propios_descuentos","ctarjeta_visa_descuentos","ctarjeta_master_descuentos","mtarjeta_master_descuentos","cforex_buy","mforex_buy","cforex_sell","ccheques_depositados_rechazados","mcheques_depositados_rechazados","ccheques_emitidos_rechazados","mcheques_emitidos_rechazados","Master_Finiciomora","Master_madelantodolares")

In [7]:
periodos_analisis  <- c( 202101, 202103 )
GLOBAL_colores <-  viridis_pal()(length( periodos_analisis ) )

In [8]:
pdf("densidades_originales.pdf")

for( campo in  campos_ordenados )
{
  cat( campo, "  " )

  graficar_campo( campo, periodos_analisis )
}

dev.off()

ctrx_quarter   cpayroll_trx   mcaja_ahorro   mpayroll   mcuentas_saldo   mpasivos_margen   mtarjeta_visa_consumo   mprestamos_personales   mrentabilidad_annual   mcuenta_corriente   mactivos_margen   active_quarter   ctarjeta_visa_transacciones   cprestamos_personales   Visa_fechaalta   mcomisiones_mantenimiento   cdescubierto_preacordado   Visa_status   Visa_mpagominimo   Master_fechaalta   cliente_edad   mrentabilidad   Master_Fvencimiento   Visa_msaldototal   numero_de_cliente   Visa_Fvencimiento   cproductos   cliente_antiguedad   ctarjeta_master   Visa_msaldopesos   ccaja_seguridad   Master_mfinanciacion_limite   mcuenta_debitos_automaticos   Master_status   mtransferencias_recibidas   chomebanking_transacciones   ccomisiones_mantenimiento   mautoservicio   ccajas_consultas   mtarjeta_master_consumo   ccomisiones_otras   Visa_cconsumos   Visa_mfinanciacion_limite   mcaja_ahorro_dolares   mttarjeta_visa_debitos_automaticos   mcomisiones_otras   mcomisiones   Visa_mpagospesos   Visa

png 
  2